In [85]:
# importing the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np
from xgboost import XGBClassifier


In [86]:
# setting the options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=np.inf)

In [87]:
X_train = pd.read_csv('X_train_V4.csv',sep=',')
X_val = pd.read_csv('X_val_V4.csv',sep=',')
y_train = pd.read_csv('y_train_V4.csv',sep=',')
y_val = pd.read_csv('y_val_V4.csv',sep=',')

#### Creating smaller stratified samples

In [88]:
X_train_sample, _, y_train_sample, _ = train_test_split(
    X_train, y_train, test_size=0.6, random_state=42, stratify=y_train
)

X_val_sample, _, y_val_sample, _ = train_test_split(
    X_val, y_val, test_size=0.6, random_state=42, stratify=y_val
)

#### Defining the model and the Pipeline

##### XGBoost

In [89]:
# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) #stratified kfold because data is imbalanced

# # Define the XGBoost model with tuned parameters
# model = XGBClassifier(
#     n_estimators=200,         # Increase the number of trees to capture more patterns
#     max_depth=6,              # Moderate depth to control complexity
#     learning_rate=0.1,        # Adjusted learning rate for optimization
#     scale_pos_weight=1,       # Initial adjustment for imbalance (optimize further if needed)
#     random_state=42,
#     use_label_encoder=False,
#     eval_metric='mlogloss'
# )

# # Adding SMOTE and UNDERSAMPLE to deal with class imbalance
# pipeline = Pipeline([
#     ('smote', SMOTE(sampling_strategy='not majority', random_state=42)),
#     ('undersample', RandomUnderSampler(sampling_strategy='auto', random_state=42)),
#     ('classifier', model)
# ])


# #0.68 on training
# #0.45 on val


In [90]:
# # Stratified K-Fold for cross-validation due to class imbalance
# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the XGBoost model with tuned parameters
model = XGBClassifier(
    n_estimators=200,         
    max_depth=6,              
    learning_rate=0.1,        
    scale_pos_weight=1,       
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'    
)

pipeline = Pipeline([
    ('classifier', model)
])

# Average Macro F1-Score on training folds: 0.6863
# Average Macro F1-Score on validation folds: 0.4185

#### Random Forest

In [91]:
# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # Stratified K-Fold devido ao desbalanceamento

# # Definição do modelo Random Forest
# model = RandomForestClassifier(
#     n_estimators=200,          # Número de árvores na floresta
#     max_depth=10,              # Profundidade máxima de cada árvore
#     min_samples_split=5,       # Mínimo de amostras necessárias para dividir um nó
#     min_samples_leaf=2,        # Mínimo de amostras necessárias em um nó folha
#     class_weight='balanced',   # Ajusta pesos para o desbalanceamento
#     random_state=42
# )

# # Average Macro F1-Score on training folds: 0.4244
# # Average Macro F1-Score on validation folds: 0.3889


#### Logistic Regression

In [92]:
# from sklearn.linear_model import LogisticRegression

# kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) 

# model = LogisticRegression()

# # Average Macro F1-Score on training folds: 
# # Average Macro F1-Score on validation folds: 

#### Applying Cross Validation on the training sample

In [93]:
train_scores = []
val_scores = []

for train_index, val_index in kf.split(X_train_sample, y_train_sample):
    # Split the data into training and validation sets for this fold
    X_fold_train, X_fold_val = X_train_sample.iloc[train_index], X_train_sample.iloc[val_index]
    y_fold_train, y_fold_val = y_train_sample.iloc[train_index], y_train_sample.iloc[val_index]
    
    # Fit the pipeline on the training fold
    pipeline.fit(X_fold_train, y_fold_train)
    
    # Predict and score on the training fold
    y_train_pred = pipeline.predict(X_fold_train)
    train_f1 = f1_score(y_fold_train, y_train_pred, average='macro')
    train_scores.append(train_f1)
    
    # Predict and score on the validation fold
    y_val_pred = pipeline.predict(X_fold_val)
    val_f1 = f1_score(y_fold_val, y_val_pred, average='macro')
    val_scores.append(val_f1)

# Calculate and print average F1-macro scores for both training and validation
print(f"Average Macro F1-Score on training folds: {np.mean(train_scores):.4f}")
print(f"Average Macro F1-Score on validation folds: {np.mean(val_scores):.4f}")



c:\Users\rodri\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:57:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rodri\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:57:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\rodri\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:57:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(sm

Average Macro F1-Score on training folds: 0.6863
Average Macro F1-Score on validation folds: 0.4185


We are ocurring in overfitting

#### Applying the model to the full dataset after we get a good F1 Macro score

In [ ]:
pipeline.fit(X_train, y_train)
y_val_pred_full = pipeline.predict(X_val)
final_score_full = f1_score(y_val, y_val_pred_full, average='macro')
print(f"Macro F1-Score on final validation set (full dataset): {final_score_full:.4f}")
